<h3>Clone repository</h3>

In [1]:
!git clone https://github.com/CLARIN-PL/personalized-nlp.git

Cloning into 'personalized-nlp'...
remote: Enumerating objects: 4368, done.
remote: Counting objects: 100% (1995/1995), done.
remote: Compressing objects: 100% (754/754), done.
remote: Total 4368 (delta 1292), reused 1794 (delta 1215), pack-reused 2373
Receiving objects: 100% (4368/4368), 46.76 MiB | 22.16 MiB/s, done.
Resolving deltas: 100% (2719/2719), done.


<h3>Download data</h3>

In [2]:
!gdown 1CyTSYita72HlfAytIPEl-2XGNOD05i5q -O /content/personalized-nlp/storage/data/unhealthy_conversations/
!gdown 1VHwrDvYx_LtFeKMaGOsD0xC5MSaGgv2H -O /content/personalized-nlp/storage/data/unhealthy_conversations/

Downloading...
From: https://drive.google.com/uc?id=1CyTSYita72HlfAytIPEl-2XGNOD05i5q
To: /content/personalized-nlp/storage/data/unhealthy_conversations/annotations_users_folds.csv
100% 12.7M/12.7M [00:00<00:00, 129MB/s]
Downloading...
From: https://drive.google.com/uc?id=1VHwrDvYx_LtFeKMaGOsD0xC5MSaGgv2H
To: /content/personalized-nlp/storage/data/unhealthy_conversations/texts_processed.csv
100% 6.24M/6.24M [00:00<00:00, 194MB/s]


<h3>Change working directory</h3>

In [3]:
%cd personalized-nlp

/content/personalized-nlp


<h3>Install required libraries</h3>

In [4]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.1/582.1 kB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [5]:
!pip uninstall torchtext -y

Found existing installation: torchtext 0.15.2
Uninstalling torchtext-0.15.2:
  Successfully uninstalled torchtext-0.15.2


<h3>Declare data module for Unhealthy Conversations dataset</h3>

In [6]:
from typing import List

import pandas as pd
import os
from pathlib import Path

from settings import DATA_DIR
from personalized_nlp.datasets.datamodule_base import BaseDataModule


class UnhealthyDataModule(BaseDataModule):
    @property
    def data_dir(self) -> Path:
        return DATA_DIR / "unhealthy_conversations"

    @property
    def annotations_file(self) -> str:
        return f"annotations_{self.stratify_folds_by}_folds.csv"

    @property
    def data_file(self) -> str:
        return f"texts_processed.csv"

    @property
    def annotation_columns(self) -> List[str]:
        return [
            "antagonize",
            "condescending",
            "dismissive",
            "generalisation",
            "generalisation_unfair",
            "healthy",
            "hostile",
            "sarcastic",
        ]

    @property
    def class_dims(self):
        return [2] * 8

    def __init__(
        self,
        gpt_personalized=False,
        **kwargs,
    ):
        self.gpt_personalized = gpt_personalized
        super().__init__(**kwargs)

        os.makedirs(self.data_dir / "embeddings", exist_ok=True)

    def prepare_data(self) -> None:
        columns_map = {"comment": "text"}
        self.data = pd.read_csv(self.data_dir / self.data_file).dropna()
        self.data = self.data.rename(columns=columns_map)

        self.annotations = pd.read_csv(self.data_dir / self.annotations_file)

        self.annotations = self.annotations.drop_duplicates(
            subset=["text_id", "annotator_id"]
        )

<h3>Import necessary libraries and modules</h3>

In [7]:
import os
from itertools import product
from operator import itemgetter

from settings import TRANSFORMER_MODEL_STRINGS

from personalized_nlp.learning.train import train_test
from settings import LOGS_DIR
from personalized_nlp.utils import seed_everything
from personalized_nlp.utils.experiments import product_kwargs
from personalized_nlp.utils.callbacks.outputs import SaveOutputsLocal
from pytorch_lightning import loggers as pl_loggers

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["WANDB_START_METHOD"] = "thread"

<h3>Run experiments</h3>

In [35]:
  wandb_project_name = "unhealthy-kk"
  datamodule_cls = UnhealthyDataModule

  datamodule_kwargs_list = product_kwargs(
      {
          "regression": [False],
          "embeddings_type": ["labse", "mpnet", "xlmr", "random", "skipgram", "bert"][
              -1:
          ],
          "limit_past_annotations_list": [None],
          "stratify_folds_by": ["users", "texts"][:1],
          "folds_num": [5],
          "batch_size": [3000],
          "test_fold": list(range(5)),
          "use_finetuned_embeddings": [False],
          "use_cuda": [True]
      }
  )
  model_kwargs_list = product_kwargs(
      {
          "huggingface_model_name": [TRANSFORMER_MODEL_STRINGS["labse"]],
          "append_annotator_ids": [False, True][:1],
          "use_cuda": [True],
          "embedding_dim": [50],
          "dp_emb": [0.25],
          "dp": [0.0],
          "hidden_dim": [100]
      }
  )
  trainer_kwargs_list = product_kwargs(
      {
          "epochs": [20],
          "lr": [0.008],
          "regression": [False],
          "use_cuda": [True],
          "model_type": list(itemgetter(0, 3)(["baseline", "onehot", "peb", "bias", "embedding"])),
      }
  )

  for datamodule_kwargs in datamodule_kwargs_list:
      seed_everything()
      data_module = datamodule_cls(**datamodule_kwargs)

      for model_kwargs, trainer_kwargs in product(
          model_kwargs_list,
          trainer_kwargs_list,
      ):
          hparams = {
              "dataset": type(data_module).__name__,
              **datamodule_kwargs,
              **model_kwargs,
              **trainer_kwargs,
          }

          logger = pl_loggers.WandbLogger(
              save_dir=str(LOGS_DIR),
              config=hparams,
              project=wandb_project_name,
              log_model=False,
          )

          train_test(
              datamodule=data_module,
              model_kwargs=model_kwargs,
              logger=logger,
              custom_callbacks=[SaveOutputsLocal(f"uc_outputs")],
              **trainer_kwargs,
          )

          logger.experiment.finish()


INFO:pytorch_lightning.utilities.seed:Global seed set to 22


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/util

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:486: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py:1938: PossibleUserWarning: The number of training samples (23) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/personalized-nlp/storage/checkpoints/prime-forest-16/epoch=14-step=360.ckpt
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/personalized-nlp/storage/checkpoints/prime-forest-16/epoch=14-step=360.ckpt
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:486: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃              Test metric               ┃              DataLoader 0              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        test_accuracy_antagonize        │           0.8606348037719727           │
│      test_accuracy_condescending       │           0.8588106632232666           │
│        test_accuracy_dismissive        │           0.893469512462616            │
│      test_accuracy_generalisation      │           0.9073330760002136           │
│  test_accuracy_generalisation_unfair   │           0.9177307486534119           │
│         test_accuracy_healthy          │           0.8095585703849792           │
│         test_accuracy_hostile          │           0.8693907260894775           │
│        test_accuracy_sarcastic         │           0.8405691385269165           │
│          test_f1_antagonize_0          │           0.9202505350112915           │
│          test_f1_antagonize_1          │          0.44797688722610474           │
│        test_f1_condescending_0         │           0.915631115436554            │
│        test_f1_condescending_1         │           0.5675977468490601           │
│          test_f1_dismissive_0          │           0.940794825553894            │
│          test_f1_dismissive_1          │          0.46909090876579285           │
│        test_f1_generalisation_0        │           0.9502155780792236           │
│        test_f1_generalisation_1        │          0.33157894015312195           │
│    test_f1_generalisation_unfair_0     │           0.9562857151031494           │
│    test_f1_generalisation_unfair_1     │           0.3029366433620453           │
│           test_f1_healthy_0            │           0.6463414430618286           │
│           test_f1_healthy_1            │           0.8696954846382141           │
│           test_f1_hostile_0            │           0.9261550903320312           │
│           test_f1_hostile_1            │          0.43533122539520264           │
│          test_f1_sarcastic_0           │           0.9072777628898621           │
│          test_f1_sarcastic_1           │           0.4317295253276825           │
│               test_loss                │           2.384270668029785            │
│        test_macro_f1_antagonize        │           0.6841137409210205           │
│      test_macro_f1_condescending       │           0.7416144609451294           │
│        test_macro_f1_dismissive        │           0.7049428820610046           │
│      test_macro_f1_generalisation      │           0.640897274017334            │
│  test_macro_f1_generalisation_unfair   │           0.6296111941337585           │
│         test_macro_f1_healthy          │           0.7580184936523438           │
│         test_macro_f1_hostile          │           0.6807431578636169           │
│        test_macro_f1_sarcastic         │           0.6695036292076111           │
│      test_precision_antagonize_0       │           0.8763419389724731           │
│      test_precision_antagonize_1       │           0.6858407258987427           │
│     test_precision_condescending_0     │           0.8760951161384583           │
│     test_precision_condescending_1     │           0.7383720874786377           │
│      test_precision_dismissive_0       │            0.91554856300354            │
│      test_precision_dismissive_1       │           0.6231883764266968           │
│    test_precision_generalisation_0     │           0.9206228852272034           │
│    test_precision_generalisation_1     │           0.5833333134651184           │
│ test_precision_generalisation_unfair_0 │           0.9284773468971252           │
│ test_precision_generalisation_unfair_1 │           0.5798816680908203           │
│        test_precision_healthy_0        │           0.7547468543052673           │
│        test_precision_healthy_1        │          

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████▆
test_accuracy_antagonize,▁
test_accuracy_condescending,▁
test_accuracy_dismissive,▁
test_accuracy_generalisation,▁
test_accuracy_generalisation_unfair,▁
test_accuracy_healthy,▁
test_accuracy_hostile,▁
test_accuracy_sarcastic,▁
test_f1_antagonize_0,▁
test_f1_antagonize_1,▁


<h3>Download results from Weights&Biases</h3>

In [36]:
import pandas as pd
from personalized_nlp.utils.wandb_download import get_wandb_project_rows

USERNAME = "persemo"
result = get_wandb_project_rows(f"{USERNAME}/{wandb_project_name}")

macro_f1_cols = [col for col in result if 'macro_f1' in col]
accuracy_cols = [col for col in result if 'accuracy' in col]


<h3>Analyze results</h3>

In [44]:
macro_f1_df = pd.DataFrame({model_type: result[result['model_type'] == model_type][macro_f1_cols]
              .mean(axis=0).round(4).tolist() for model_type in result.model_type.unique().tolist()})
macro_f1_df.index = macro_f1_cols
macro_f1_df['diff'] = macro_f1_df['bias'] - macro_f1_df['baseline']

macro_f1_df

,bias,baseline,diff
test_macro_f1_generalisation_unfair,0.6236,0.5188,0.1048
test_macro_f1_dismissive,0.7009,0.5684,0.1325
test_macro_f1_hostile,0.6869,0.5932,0.0937
test_macro_f1_sarcastic,0.6346,0.4833,0.1513
test_macro_f1_condescending,0.7233,0.5789,0.1444
test_macro_f1_generalisation,0.6298,0.5370,0.0928
test_macro_f1_antagonize,0.7097,0.5777,0.1320
test_macro_f1_healthy,0.7535,0.5367,0.2168


In [45]:
acc_df = pd.DataFrame({model_type: result[result['model_type'] == model_type][accuracy_cols]
              .mean(axis=0).round(4).tolist() for model_type in result.model_type.unique().tolist()})
acc_df.index = accuracy_cols
acc_df['diff'] = acc_df['bias'] - acc_df['baseline']

acc_df

,bias,baseline,diff
test_accuracy_hostile,0.9003,0.8929,0.0074
test_accuracy_sarcastic,0.8589,0.8439,0.0150
test_accuracy_dismissive,0.9013,0.8869,0.0144
test_accuracy_healthy,0.8500,0.7826,0.0674
test_accuracy_generalisation_unfair,0.9293,0.9234,0.0059
test_accuracy_generalisation,0.9229,0.9166,0.0063
test_accuracy_condescending,0.8699,0.8410,0.0289
test_accuracy_antagonize,0.8786,0.8564,0.0222
